In [1]:
!pip install wrds 
import wrds 
import pandas as pd

In [2]:
db = wrds.Connection()

Loading library list...
Done


In [4]:
query_funda = """
SELECT
    gvkey,
    conm,
    tic,
    cusip,
    cik,
    datadate,
    fyear,
    at,     -- total assets
    lt,     -- total liabilities
    ceq,    -- common equity
    sale,   -- net sales / turnover
    revt,   -- total revenue
    ni      -- net income
FROM comp_na_daily_all.funda
WHERE indfmt = 'INDL'
  AND datafmt = 'STD'
  AND consol = 'C'
  AND popsrc IN ('D','I')
  AND fic IN ('USA','CAN')
  AND datadate BETWEEN '2000-01-01' AND '2024-12-31'
"""
df_funda = db.raw_sql(query_funda, date_cols=['datadate'])

In [5]:
print(df_funda.shape) 
print(df_funda.head())

(261599, 13)
    gvkey      conm  tic      cusip         cik   datadate  fyear       at  \
0  001004  AAR CORP  AIR  000361105  0000001750 2000-05-31   1999  740.998   
1  001004  AAR CORP  AIR  000361105  0000001750 2001-05-31   2000  701.854   
2  001004  AAR CORP  AIR  000361105  0000001750 2002-05-31   2001  710.199   
3  001004  AAR CORP  AIR  000361105  0000001750 2003-05-31   2002  686.621   
4  001004  AAR CORP  AIR  000361105  0000001750 2004-05-31   2003  709.292   

        lt      ceq      sale      revt      ni  
0  401.483  339.515  1024.333  1024.333  35.163  
1  361.642  340.212   874.255   874.255  18.531  
2  399.964  310.235   638.721   638.721 -58.939  
3  391.633  294.988   606.337   606.337  -12.41  
4  407.608  301.684   651.958   651.958   3.504  


In [6]:
print(df_funda.shape)
print(df_funda['tic'].nunique(), "unique tickers")
print(df_funda['gvkey'].nunique(), "unique firms")
df_funda.head(10)

(261599, 13)
28578 unique tickers
28599 unique firms


,gvkey,conm,tic,cusip,cik,datadate,fyear,at,lt,ceq,sale,revt,ni
0,001004,AAR CORP,AIR,000361105,0000001750,2000-05-31,1999,740.998,401.483,339.515,1024.333,1024.333,35.163
1,001004,AAR CORP,AIR,000361105,0000001750,2001-05-31,2000,701.854,361.642,340.212,874.255,874.255,18.531
2,001004,AAR CORP,AIR,000361105,0000001750,2002-05-31,2001,710.199,399.964,310.235,638.721,638.721,-58.939
3,001004,AAR CORP,AIR,000361105,0000001750,2003-05-31,2002,686.621,391.633,294.988,606.337,606.337,-12.41
4,001004,AAR CORP,AIR,000361105,0000001750,2004-05-31,2003,709.292,407.608,301.684,651.958,651.958,3.504
5,001004,AAR CORP,AIR,000361105,0000001750,2005-05-31,2004,732.23,417.486,314.744,747.848,747.848,15.453
6,001004,AAR CORP,AIR,000361105,0000001750,2006-05-31,2005,978.819,556.102,422.717,897.284,897.284,35.163
7,001004,AAR CORP,AIR,000361105,0000001750,2007-05-31,2006,1067.633,573.39,494.243,1061.169,1061.169,58.66
8,001004,AAR CORP,AIR,000361105,0000001750,2008-05-31,2007,1362.01,776.755,585.255,1384.919,1384.919,75.144
9,001004,AAR CORP,AIR,000361105,0000001750,2009-05-31,2008,1377.511,720.616,656.895,1423.976,1423.976,78.651


In [7]:
query_exec = """
SELECT
    gvkey,
    CAST(year AS int) AS fyear,
    execid,
    exec_fullname,
    ceoann,                 -- Y/N
    titleann,
    salary,
    bonus,
    tdc1,                   -- preferred total direct comp
    tdc2
FROM comp_execucomp.anncomp
WHERE year BETWEEN 2000 AND 2024
"""
df_exec = db.raw_sql(query_exec)

In [8]:
print(df_exec.shape)
print(df_exec.head())

(281860, 10)
    gvkey  fyear execid    exec_fullname ceoann         titleann   salary  \
0  001004   2000  09249  David P. Storch    CEO  president & CEO    676.0   
1  001004   2001  09249  David P. Storch    CEO  president & CEO    665.4   
2  001004   2002  09249  David P. Storch    CEO  president & CEO  661.466   
3  001004   2003  09249  David P. Storch    CEO  president & CEO    661.4   
4  001004   2004  09249  David P. Storch    CEO  president & CEO    695.4   

     bonus      tdc1      tdc2  
0    270.0  3910.003  1666.589  
1      0.0  2047.286     766.6  
2      0.0  2080.489   812.566  
3    496.1  4325.731  3534.879  
4  591.838  7045.048  5606.776  


In [9]:

df_ceo = df_exec[df_exec['ceoann'].str.upper() == 'CEO'].copy()

In [10]:

df_ceo = (df_ceo.sort_values(['gvkey','fyear','tdc1'], na_position='last')
                  .groupby(['gvkey','fyear'], as_index=False)
                  .last())

In [11]:

merged = df_funda.merge(df_ceo, on=['gvkey','fyear'], how='left')

In [12]:

print("Firm-years in Compustat:", df_funda.shape[0])
print("Firm-years matched to CEO:", merged['exec_fullname'].notna().sum())

Firm-years in Compustat: 261599
Firm-years matched to CEO: 47804


In [13]:
merged.head(10)

,gvkey,conm,tic,cusip,cik,datadate,fyear,at,lt,ceq,...,revt,ni,execid,exec_fullname,ceoann,titleann,salary,bonus,tdc1,tdc2
0,001004,AAR CORP,AIR,000361105,0000001750,2000-05-31,1999,740.998,401.483,339.515,...,1024.333,35.163,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,001004,AAR CORP,AIR,000361105,0000001750,2001-05-31,2000,701.854,361.642,340.212,...,874.255,18.531,09249,David P. Storch,CEO,president & CEO,676.0,270.0,3910.003,1666.589
2,001004,AAR CORP,AIR,000361105,0000001750,2002-05-31,2001,710.199,399.964,310.235,...,638.721,-58.939,09249,David P. Storch,CEO,president & CEO,665.4,0.0,2047.286,766.6
3,001004,AAR CORP,AIR,000361105,0000001750,2003-05-31,2002,686.621,391.633,294.988,...,606.337,-12.41,09249,David P. Storch,CEO,president & CEO,661.466,0.0,2080.489,812.566
4,001004,AAR CORP,AIR,000361105,0000001750,2004-05-31,2003,709.292,407.608,301.684,...,651.958,3.504,09249,David P. Storch,CEO,president & CEO,661.4,496.1,4325.731,3534.879
5,001004,AAR CORP,AIR,000361105,0000001750,2005-05-31,2004,732.23,417.486,314.744,...,747.848,15.453,09249,David P. Storch,CEO,president & CEO,695.4,591.838,7045.048,5606.776
6,001004,AAR CORP,AIR,000361105,0000001750,2006-05-31,2005,978.819,556.102,422.717,...,897.284,35.163,09249,David P. Storch,CEO,"chairman, president & CEO",716.6,1041.051,6568.409,12728.39
7,001004,AAR CORP,AIR,000361105,0000001750,2007-05-31,2006,1067.633,573.39,494.243,...,1061.169,58.66,09249,David P. Storch,CEO,"chairman, president & CEO",741.5,0.0,6787.1,12855.4
8,001004,AAR CORP,AIR,000361105,0000001750,2008-05-31,2007,1362.01,776.755,585.255,...,1384.919,75.144,09249,David P. Storch,CEO,chairman & CEO,768.248,0.0,2453.37,8326.946
9,001004,AAR CORP,AIR,000361105,0000001750,2009-05-31,2008,1377.511,720.616,656.895,...,1423.976,78.651,09249,David P. Storch,CEO,chairman & CEO,791.295,0.0,3313.996,3313.996


In [14]:
merged.columns.tolist()


['gvkey',
 'conm',
 'tic',
 'cusip',
 'cik',
 'datadate',
 'fyear',
 'at',
 'lt',
 'ceq',
 'sale',
 'revt',
 'ni',
 'execid',
 'exec_fullname',
 'ceoann',
 'titleann',
 'salary',
 'bonus',
 'tdc1',
 'tdc2']

In [15]:
import pandas as pd
pd.set_option('display.max_rows', 50)      # adjust rows
pd.set_option('display.max_columns', None) # show all columns
merged.head(50)

,gvkey,conm,tic,cusip,cik,datadate,fyear,at,lt,ceq,sale,revt,ni,execid,exec_fullname,ceoann,titleann,salary,bonus,tdc1,tdc2
0,001004,AAR CORP,AIR,000361105,0000001750,2000-05-31,1999,740.998,401.483,339.515,1024.333,1024.333,35.163,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,001004,AAR CORP,AIR,000361105,0000001750,2001-05-31,2000,701.854,361.642,340.212,874.255,874.255,18.531,09249,David P. Storch,CEO,president & CEO,676.0,270.0,3910.003,1666.589
2,001004,AAR CORP,AIR,000361105,0000001750,2002-05-31,2001,710.199,399.964,310.235,638.721,638.721,-58.939,09249,David P. Storch,CEO,president & CEO,665.4,0.0,2047.286,766.6
3,001004,AAR CORP,AIR,000361105,0000001750,2003-05-31,2002,686.621,391.633,294.988,606.337,606.337,-12.41,09249,David P. Storch,CEO,president & CEO,661.466,0.0,2080.489,812.566
4,001004,AAR CORP,AIR,000361105,0000001750,2004-05-31,2003,709.292,407.608,301.684,651.958,651.958,3.504,09249,David P. Storch,CEO,president & CEO,661.4,496.1,4325.731,3534.879
5,001004,AAR CORP,AIR,000361105,0000001750,2005-05-31,2004,732.23,417.486,314.744,747.848,747.848,15.453,09249,David P. Storch,CEO,president & CEO,695.4,591.838,7045.048,5606.776
6,001004,AAR CORP,AIR,000361105,0000001750,2006-05-31,2005,978.819,556.102,422.717,897.284,897.284,35.163,09249,David P. Storch,CEO,"chairman, president & CEO",716.6,1041.051,6568.409,12728.39
7,001004,AAR CORP,AIR,000361105,0000001750,2007-05-31,2006,1067.633,573.39,494.243,1061.169,1061.169,58.66,09249,David P. Storch,CEO,"chairman, president & CEO",741.5,0.0,6787.1,12855.4
8,001004,AAR CORP,AIR,000361105,0000001750,2008-05-31,2007,1362.01,776.755,585.255,1384.919,1384.919,75.144,09249,David P. Storch,CEO,chairman & CEO,768.248,0.0,2453.37,8326.946
9,001004,AAR CORP,AIR,000361105,0000001750,2009-05-31,2008,1377.511,720.616,656.895,1423.976,1423.976,78.651,09249,David P. Storch,CEO,chairman & CEO,791.295,0.0,3313.996,3313.996


In [16]:
# Export to CSV
merged.to_csv("na_funda_execucomp_2000_2024.csv", index=False)

print("File saved as: na_funda_execucomp_2000_2024.csv")

File saved as: na_funda_execucomp_2000_2024.csv
